In [1]:
# Importar librerias
import pandas as pd
import numpy as np

In [2]:
# Ruta del archivo en el entorno de Colab
file_path = 'serie_cgi_10_24.xls'

# Leer las pestañas específicas y crear DataFrames
df_vab_pb = pd.read_excel(file_path, sheet_name='VAB_pb')
df_puestos = pd.read_excel(file_path, sheet_name='Puestos')
df_horas = pd.read_excel(file_path, sheet_name='Horas')

In [3]:
# Filas a conservar (de índice 10 a 29 porque Python usa índices base 0)
rows_to_keep = range(9, 29)

# Nuevos nombres de las columnas
new_columns = [
    'COD', 'Sector', '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2016TOT',
    '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2017TOT', '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2018TOT',
    '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2019TOT', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020TOT',
    '2021Q1', '2021Q2', '2021Q3', '2021Q4', '2021TOT', '2022Q1', '2022Q2', '2022Q3', '2022Q4', '2022TOT',
    '2023Q1', '2023Q2', '2023Q3', '2023Q4', '2023TOT', '201624Q1', '2024Q2'
]

# Índices de las columnas a eliminar
columns_to_drop = [7, 13, 19, 25, 31, 37, 43, 49]

# Función para procesar cada DataFrame
def process_dataframe(df):
    # Eliminar columnas específicas por índice
    df = df.drop(df.columns[columns_to_drop], axis=1)
    # Conservar solo las filas especificadas
    df = df.iloc[rows_to_keep]
    # Ajustar el número de columnas antes de renombrar
    df = df.iloc[:, :len(new_columns)]  # Seleccionar hasta el número esperado de columnas
    # Renombrar las columnas
    df.columns = new_columns
    return df

# Aplicar la función a cada DataFrame
df_vab_pb = process_dataframe(df_vab_pb)
df_puestos = process_dataframe(df_puestos)
df_horas = process_dataframe(df_horas)

In [4]:
# Función para dividir dos DataFrames, excluyendo las columnas no numéricas (COD y Sector)
def divide_dataframes(df_numerator, df_denominator):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_numerator.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar solo las columnas numéricas
    df_numerator_numeric = df_numerator.iloc[:, 2:]
    df_denominator_numeric = df_denominator.iloc[:, 2:]
    # Realizar la división
    result_numeric = df_numerator_numeric.div(df_denominator_numeric)
    # Unir las columnas no numéricas con el resultado de la división
    result = pd.concat([cod_sector, result_numeric], axis=1)
    # Reemplazar infinitos con NaN (por si hay divisiones por cero)
    result.replace([np.inf, -np.inf], np.nan, inplace=True)
    return result

# Crear el DataFrame "productividad_puestos"
productividad_puestos = divide_dataframes(df_vab_pb, df_puestos)

# Crear el DataFrame "productividad_horas"
productividad_horas = divide_dataframes(df_vab_pb, df_horas)

<ipython-input-4-350d43fdb2b1>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)
<ipython-input-4-350d43fdb2b1>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)


In [5]:
# Clasificar los sectores en "Baja", "Media" y "Alta"
sectores_baja = ['A', 'B', 'C']
sectores_media = ['D', 'E', 'F', 'I', 'M', 'MM', 'MMM', 'N', 'NN', 'NNN']
sectores_alta = ['G', 'H', 'J', 'K', 'L', 'O', 'P']

# Función para clasificar los sectores
def clasificar_sector(cod):
    if pd.isna(cod):  # Manejar casos donde el código sea NaN
        return 'Desconocido'
    elif cod in sectores_baja:
        return 'Baja'
    elif cod in sectores_media:
        return 'Media'
    elif cod in sectores_alta:
        return 'Alta'
    else:
        return 'Desconocido'

# Aplicar la clasificación a los DataFrames
df_vab_pb['Clasificacion'] = df_vab_pb['COD'].apply(clasificar_sector)
df_horas['Clasificacion'] = df_horas['COD'].apply(clasificar_sector)

# Función para ajustar las horas trabajadas según la clasificación
def ajustar_horas(df):
    # Crear una copia del DataFrame para no modificar el original
    df_ajustado = df.copy()

    # Convertir las columnas numéricas a tipo float, excluyendo 'COD', 'Sector' y 'Clasificacion'
    columnas_numericas = df_ajustado.columns[2:-1]  # Asegurar el rango correcto
    df_ajustado[columnas_numericas] = df_ajustado[columnas_numericas].apply(pd.to_numeric, errors='coerce')

    # Función para aplicar el ajuste a una fila
    def ajustar_fila(row):
        clasificacion = row['Clasificacion']
        # Convertir horas a minutos
        row[columnas_numericas] = row[columnas_numericas] * 60

        # Aplicar el factor de ajuste en minutos según la clasificación
        if clasificacion == 'Baja':
            factor_ajuste = 1  # 1 minuto cada 14.4 minutos
            divisor = 21.6
        elif clasificacion == 'Media':
            factor_ajuste = 2  # 2 minutos cada 18 minutos
            divisor = 18
        elif clasificacion == 'Alta':
            factor_ajuste = 3  # 3 minutos cada 21.6 minutos
            divisor = 14.4
        else:
            factor_ajuste = 0
            divisor = 1  # Valor por defecto (no debería usarse)

        # Ajustar las columnas numéricas
        row[columnas_numericas] = row[columnas_numericas] * (1 + factor_ajuste / divisor)

        # Convertir de nuevo a horas
        row[columnas_numericas] = row[columnas_numericas] / 60
        return row

    # Aplicar la función fila por fila
    df_ajustado = df_ajustado.apply(ajustar_fila, axis=1)
    return df_ajustado

# Ajustar las horas trabajadas en df_horas
df_horas_ajustado = ajustar_horas(df_horas)

# Función para dividir dos DataFrames, excluyendo las columnas no numéricas (COD y Sector)
def divide_dataframes(df_numerator, df_denominator):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_numerator.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar solo las columnas numéricas
    columnas_numericas = df_numerator.columns[2:-1]  # Excluir 'Clasificacion' y no numéricas
    df_numerator_numeric = df_numerator[columnas_numericas]
    df_denominator_numeric = df_denominator[columnas_numericas]

    # Realizar la división (manejar posibles divisiones por cero)
    result_numeric = df_numerator_numeric.div(df_denominator_numeric.replace(0, np.nan))

    # Unir las columnas no numéricas con el resultado de la división
    result = pd.concat([cod_sector, result_numeric], axis=1)
    # Reemplazar infinitos con NaN (por si hay divisiones por cero)
    result.replace([np.inf, -np.inf], np.nan, inplace=True)
    return result

# Crear el DataFrame "productividad_horas_distraccion"
productividad_horas_distraccion = divide_dataframes(df_vab_pb, df_horas_ajustado)

productividad_horas_distraccion

<ipython-input-5-d2fd70bfe66d>:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result.replace([np.inf, -np.inf], np.nan, inplace=True)


,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.161580,0.431028,0.177870,0.163319,0.228375,0.204639,0.374130,0.201597,...,2.026265,1.851589,2.389217,2.815004,4.982709,4.624793,6.630994,4.895946,9.736560,21.075117
10,B,Pesca,0.187709,0.379519,0.482593,0.305531,0.338195,0.243476,0.449751,0.489992,...,3.909749,2.163221,3.019308,2.738058,6.032378,8.959848,5.523627,5.811876,13.540434,28.769589
11,C,Explotación de minas y canteras,1.122098,1.129353,1.148618,1.189063,1.146964,1.368356,1.310724,1.379269,...,10.721318,14.901699,10.559429,17.266452,19.910765,28.018089,44.486689,27.480518,76.307180,80.593895
12,D,Industria manufacturera,0.208170,0.233490,0.236207,0.252380,0.232777,0.255670,0.301431,0.292486,...,3.064819,3.299245,2.679707,3.632909,5.362869,6.671398,8.990633,6.160451,13.393054,17.901616
13,E,"Electricidad, gas y agua",0.520372,0.582622,0.665306,0.549736,0.581623,0.691651,0.769012,0.971045,...,3.198987,2.706855,2.749301,4.763256,5.890734,8.061190,8.908115,7.037368,21.649533,35.915222
14,F,Construcción,0.101431,0.099779,0.109311,0.107979,0.104723,0.136186,0.138282,0.151461,...,0.942759,1.115825,0.908042,1.587619,1.681217,2.285326,2.875382,2.112343,4.480764,5.294576
15,G,"Comercio mayorista, minorista y reparaciones",0.126528,0.141723,0.148145,0.152052,0.142389,0.167813,0.181231,0.189108,...,1.705608,2.172283,1.600459,2.261474,3.008396,3.725473,5.383494,3.600469,7.945358,9.506200
16,H,Hoteles y restaurantes,0.118552,0.117823,0.133409,0.143826,0.128544,0.159454,0.169016,0.176314,...,1.070076,1.388627,1.086340,2.093871,2.240380,2.659415,3.764878,2.722161,6.781308,7.201240
17,I,"Transporte, almacenamiento y comunicaciones",0.147778,0.166264,0.165953,0.183382,0.165855,0.205705,0.233340,0.218852,...,1.531793,1.927408,1.503025,2.132391,2.644307,3.490919,4.447180,3.181090,7.556249,10.512287
18,J,Intermediación financiera,0.537527,0.509257,0.542968,0.638323,0.556896,0.702064,0.732561,0.698929,...,3.568829,3.741602,3.548464,4.744211,5.194839,4.588458,6.467224,5.237718,30.988535,38.094280


In [6]:
# Crear el DataFrame "merma_productividad" restando los valores entre los DataFrames
def calcular_merma_productividad(df_original, df_distraccion):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_original.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_original.columns[2:]

    # Realizar la resta solo en las columnas numéricas
    df_original_numeric = df_original[columnas_numericas]
    df_distraccion_numeric = df_distraccion[columnas_numericas]

    # Calcular la merma restando los valores
    merma_numeric = df_original_numeric - df_distraccion_numeric

    # Unir las columnas no numéricas con el resultado de la resta
    merma = pd.concat([cod_sector, merma_numeric], axis=1)

    # Manejar valores NaN que puedan generarse en la operación
    merma.fillna(0, inplace=True)

    return merma

# Calcular el DataFrame "merma_productividad"
merma_productividad = calcular_merma_productividad(productividad_horas, productividad_horas_distraccion)

merma_productividad

,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.007481,0.019955,0.008235,0.007561,0.010573,0.009474,0.017321,0.009333,...,0.093809,0.085722,1.106119e-01,0.130324,2.306810e-01,2.141108e-01,3.069904e-01,2.266642e-01,0.450767,0.975700
10,B,Pesca,0.008690,0.017570,0.022342,0.014145,0.015657,0.011272,0.020822,0.022685,...,0.181007,0.100149,1.397828e-01,0.126762,2.792767e-01,4.148078e-01,2.557235e-01,2.690683e-01,0.626872,1.331925
11,C,Explotación de minas y canteras,0.051949,0.052285,0.053177,0.055049,0.053100,0.063350,0.060682,0.063855,...,0.496357,0.689893,4.888624e-01,0.799373,9.217947e-01,1.297134e+00,2.059569e+00,1.272246e+00,3.532740,3.731199
12,D,Industria manufacturera,0.023130,0.025943,0.026245,0.028042,0.025864,0.028408,0.033492,0.032498,...,0.340535,0.366583,2.977452e-01,0.403657,5.958744e-01,7.412665e-01,9.989592e-01,6.844946e-01,1.488117,1.989068
13,E,"Electricidad, gas y agua",0.057819,0.064736,0.073923,0.061082,0.064625,0.076850,0.085446,0.107894,...,0.355443,0.300762,3.054779e-01,0.529251,6.545260e-01,8.956877e-01,9.897906e-01,7.819297e-01,2.405504,3.990580
14,F,Construcción,0.011270,0.011087,0.012146,0.011998,0.011636,0.015132,0.015365,0.016829,...,0.104751,0.123981,1.008935e-01,0.176402,1.868019e-01,2.539251e-01,3.194869e-01,2.347048e-01,0.497863,0.588286
15,G,"Comercio mayorista, minorista y reparaciones",0.026360,0.029526,0.030864,0.031677,0.029664,0.034961,0.037757,0.039397,...,0.355335,0.452559,3.334290e-01,0.471140,6.267493e-01,7.761403e-01,1.121561e+00,7.500976e-01,1.655283,1.980458
16,H,Hoteles y restaurantes,0.024698,0.024546,0.027794,0.029964,0.026780,0.033220,0.035212,0.036732,...,0.222932,0.289297,2.263208e-01,0.436223,4.667457e-01,5.540449e-01,7.843497e-01,5.671168e-01,1.412773,1.500258
17,I,"Transporte, almacenamiento y comunicaciones",0.016420,0.018474,0.018439,0.020376,0.018428,0.022856,0.025927,0.024317,...,0.170199,0.214156,1.670028e-01,0.236932,2.938119e-01,3.878799e-01,4.941311e-01,3.534544e-01,0.839583,1.168032
18,J,Intermediación financiera,0.111985,0.106095,0.113118,0.132984,0.116020,0.146263,0.152617,0.145610,...,0.743506,0.779500,7.392633e-01,0.988377,1.082258e+00,9.559288e-01,1.347338e+00,1.091191e+00,6.455945,7.936308


In [7]:
# Crear el DataFrame "merma_productividad_valor" multiplicando los valores de "merma_productividad" por "df_horas"
def calcular_merma_productividad_valor(df_merma, df_horas):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_merma.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_merma.columns[2:]

    # Realizar la multiplicación solo en las columnas numéricas
    df_merma_numeric = df_merma[columnas_numericas]
    df_horas_numeric = df_horas[columnas_numericas]

    # Calcular el valor de la merma multiplicando los valores
    merma_valor_numeric = df_merma_numeric * df_horas_numeric

    # Unir las columnas no numéricas con el resultado de la multiplicación
    merma_valor = pd.concat([cod_sector, merma_valor_numeric], axis=1)

    # Manejar valores NaN que puedan generarse en la operación
    merma_valor.fillna(0, inplace=True)

    return merma_valor

# Calcular el DataFrame "merma_productividad_valor"
merma_productividad_valor = calcular_merma_productividad_valor(merma_productividad, df_horas)

merma_productividad_valor

,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",13660.62646,37953.935275,17774.026597,17951.079966,21834.917075,17718.75445,33958.413672,19908.787845,...,205407.159225,206655.349152,232982.490518,245601.842857,444215.002108,472125.073968,770101.472803,483010.847934,871895.125344,1910738.5633
10,B,Pesca,530.312539,1084.48446,1355.552619,921.569911,972.979882,755.363581,1444.503223,1607.580014,...,14494.82271,6329.145545,9793.61247,9308.789715,18886.951932,30473.751407,18143.114751,19203.151951,43432.994565,91157.244207
11,C,Explotación de minas y canteras,11192.720068,12068.83829,11749.47556,11843.371125,11713.601261,13263.037348,12932.8938,13932.604803,...,126489.506182,161515.383721,116929.980368,203265.383547,243067.8288,353040.10719,538563.434901,334484.18861,880540.589671,1005210.028241
12,D,Industria manufacturera,96714.038096,109684.10266,116413.412058,121114.966355,110981.629792,116587.396727,136204.129717,144857.083355,...,1578263.776098,1666525.635951,1347393.304082,1863104.058529,2674667.34367,3401420.469734,4553811.298581,3123250.792629,6583542.008539,8501618.369019
13,E,"Electricidad, gas y agua",9809.387187,12868.120318,14747.321198,12454.049057,12469.71944,13737.110853,17691.124789,21811.576803,...,77073.171491,78869.277581,68096.101683,104248.874698,134568.078768,196677.186333,242783.892366,169569.508041,451147.206358,667542.234068
14,F,Construcción,29239.822263,29800.325436,34029.524272,33092.827724,31540.624924,40610.437862,41764.31495,51017.95271,...,344856.255886,378815.516841,304540.877572,531807.222758,595887.696915,871298.319896,989655.050745,747162.072578,1330240.813822,1626814.792557
15,G,"Comercio mayorista, minorista y reparaciones",161897.387277,187275.476562,204136.140538,208386.780332,190423.946177,216157.831909,241415.489597,275429.423388,...,2644739.307389,3063119.501566,2378873.539628,3426001.628951,4599639.682721,5967277.630505,8253307.611026,5561556.638301,12207960.47885,14907668.069598
16,H,Hoteles y restaurantes,27528.496116,25542.445902,30109.337398,33375.386384,29138.91645,37588.356153,35874.210496,41425.164031,...,265122.371331,333425.5533,255722.763766,521865.274366,488863.222624,699111.599365,1004529.767262,678592.465904,1764562.205485,1619597.598486
17,I,"Transporte, almacenamiento y comunicaciones",38321.778089,45321.94771,44865.290281,47600.516304,44027.383096,54313.653193,61652.472435,60581.372482,...,396007.007637,475213.319956,385356.082945,558609.161909,709161.266582,878973.532225,1196599.516426,835835.869286,1993579.023592,2814137.592612
18,J,Intermediación financiera,48434.740872,51951.119497,57844.113308,63410.679051,55410.163182,67205.179866,69620.597577,72623.888286,...,363777.762309,372108.326706,348020.620519,433153.5441,547368.727692,505685.898776,637144.819788,530838.247589,2782158.048718,3833466.878496


In [8]:
# Crear el DataFrame "merma_productividad_porcentaje" dividiendo "merma_productividad_valor" sobre "df_vab_pb"
def calcular_merma_productividad_porcentaje(df_valor, df_vab):
    # Separar las columnas no numéricas (COD y Sector)
    cod_sector = df_valor.iloc[:, :2]  # Columnas 0 y 1
    # Seleccionar las columnas numéricas
    columnas_numericas = df_valor.columns[2:]

    # Realizar la división solo en las columnas numéricas
    df_valor_numeric = df_valor[columnas_numericas]
    df_vab_numeric = df_vab[columnas_numericas]

    # Calcular el porcentaje de merma productiva
    merma_porcentaje_numeric = df_valor_numeric.div(df_vab_numeric)

    # Unir las columnas no numéricas con el resultado de la división
    merma_porcentaje = pd.concat([cod_sector, merma_porcentaje_numeric], axis=1)

    # Manejar valores infinitos y nulos generados durante la división
    merma_porcentaje.replace([np.inf, -np.inf], np.nan, inplace=True)
    merma_porcentaje.fillna(0, inplace=True)

    return merma_porcentaje

# Calcular el DataFrame "merma_productividad_porcentaje"
merma_productividad_porcentaje = calcular_merma_productividad_porcentaje(merma_productividad_valor, df_vab_pb)

merma_productividad_porcentaje

<ipython-input-8-e47240fc7d35>:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merma_porcentaje.replace([np.inf, -np.inf], np.nan, inplace=True)


,COD,Sector,2016Q1,2016Q2,2016Q3,2016Q4,2016TOT,2017Q1,2017Q2,2017Q3,...,2022Q3,2022Q4,2022TOT,2023Q1,2023Q2,2023Q3,2023Q4,2023TOT,201624Q1,2024Q2
9,A,"Agricultura, ganadería, caza y silvicultura",0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,...,0.044248,0.044248,4.424779e-02,0.044248,4.424779e-02,4.424779e-02,4.424779e-02,4.424779e-02,0.044248,0.044248
10,B,Pesca,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,...,0.044248,0.044248,4.424779e-02,0.044248,4.424779e-02,4.424779e-02,4.424779e-02,4.424779e-02,0.044248,0.044248
11,C,Explotación de minas y canteras,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,0.044248,...,0.044248,0.044248,4.424779e-02,0.044248,4.424779e-02,4.424779e-02,4.424779e-02,4.424779e-02,0.044248,0.044248
12,D,Industria manufacturera,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,1.000000e-01,0.100000,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,0.100000,0.100000
13,E,"Electricidad, gas y agua",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,1.000000e-01,0.100000,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,0.100000,0.100000
14,F,Construcción,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,1.000000e-01,0.100000,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,0.100000,0.100000
15,G,"Comercio mayorista, minorista y reparaciones",0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,...,0.172414,0.172414,1.724138e-01,0.172414,1.724138e-01,1.724138e-01,1.724138e-01,1.724138e-01,0.172414,0.172414
16,H,Hoteles y restaurantes,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,...,0.172414,0.172414,1.724138e-01,0.172414,1.724138e-01,1.724138e-01,1.724138e-01,1.724138e-01,0.172414,0.172414
17,I,"Transporte, almacenamiento y comunicaciones",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,...,0.100000,0.100000,1.000000e-01,0.100000,1.000000e-01,1.000000e-01,1.000000e-01,1.000000e-01,0.100000,0.100000
18,J,Intermediación financiera,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,0.172414,...,0.172414,0.172414,1.724138e-01,0.172414,1.724138e-01,1.724138e-01,1.724138e-01,1.724138e-01,0.172414,0.172414


In [9]:
# Guardar los DataFrames en un archivo Excel con múltiples pestañas
with pd.ExcelWriter('analisis_productividad.xlsx') as writer:
    df_vab_pb.to_excel(writer, sheet_name='vab_pb', index=False)
    df_puestos.to_excel(writer, sheet_name='puestos', index=False)
    df_horas.to_excel(writer, sheet_name='horas', index=False)
    productividad_puestos.to_excel(writer, sheet_name='productividad_puestos', index=False)
    productividad_horas.to_excel(writer, sheet_name='productividad_horas', index=False)
    productividad_horas_distraccion.to_excel(writer, sheet_name='productividad_horas_distraccion', index=False)
    merma_productividad.to_excel(writer, sheet_name='merma_productividad', index=False)
    merma_productividad_valor.to_excel(writer, sheet_name='merma_productividad_valor', index=False)
    merma_productividad_porcentaje.to_excel(writer, sheet_name='merma_productividad_porcentaje', index=False)